In [1]:
# Question: Multivariate Outlier Detection Using Mahalanobis Distance
# Description: Implement Mahalanobis distance to detect multivariate outliers in a dataset.



In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from numpy.linalg import inv
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
def detect_outliers_mahalanobis(X, threshold_percent=95):
    mean_vector = np.mean(X, axis=0)
    cov_matrix = np.cov(X, rowvar=False)
    inv_cov_matrix = inv(cov_matrix)
    mahal_dist = []
    for i in range(X.shape[0]):
        mahal_dist.append(mahalanobis(X[i], mean_vector, inv_cov_matrix))
    mahal_dist = np.array(mahal_dist)
    threshold = np.percentile(mahal_dist, threshold_percent)
    outliers = np.where(mahal_dist > threshold)[0]
    return mahal_dist, threshold, outliers
np.random.seed(42)
n_samples = 500
n_features = 2
mean = [0, 0]
cov = [[1, 0.8], [0.8, 1]]
X = np.random.multivariate_normal(mean, cov, n_samples)
outliers_idx = np.random.randint(0, n_samples, 15)
X[outliers_idx] = X[outliers_idx] + np.random.normal(0, 5, (15, n_features))
mahal_dist, threshold, outliers = detect_outliers_mahalanobis(X, threshold_percent=97.5)
plt.figure(figsize=(12, 10))
plt.subplot(2, 2, 1)
plt.scatter(X[:, 0], X[:, 1], alpha=0.5)
plt.scatter(X[outliers, 0], X[outliers, 1], color='red')
plt.title(f'Outliers using Mahalanobis Distance (threshold: {threshold:.2f})')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.subplot(2, 2, 2)
plt.hist(mahal_dist, bins=30)
plt.axvline(x=threshold, color='r', linestyle='--')
plt.title('Distribution of Mahalanobis Distances')
plt.xlabel('Mahalanobis Distance')
plt.ylabel('Frequency')
results = pd.DataFrame({
    'Feature1': X[:, 0],
    'Feature2': X[:, 1],
    'MahalanobisDistance': mahal_dist,
    'IsOutlier': np.isin(np.arange(n_samples), outliers)
})
plt.subplot(2, 2, 3)
sns.scatterplot(data=results, x='Feature1', y='Feature2', hue='MahalanobisDistance', 
                palette='viridis', alpha=0.7)
plt.title('Heatmap of Mahalanobis Distances')
plt.subplot(2, 2, 4)
df = results.copy()
df['CategoricalOutlier'] = np.where(df['IsOutlier'], 'Outlier', 'Normal')
sns.scatterplot(data=df, x='Feature1', y='Feature2', hue='CategoricalOutlier', 
                palette={'Outlier': 'red', 'Normal': 'blue'}, alpha=0.7)
plt.title('Detected Outliers')
plt.tight_layout()
plt.show()
print(f"Detected {len(outliers)} outliers out of {n_samples} samples ({len(outliers)/n_samples*100:.2f}%)")
print("\nTop 10 points with highest Mahalanobis distance:")
print(results.sort_values('MahalanobisDistance', ascending=False).head(10)[['Feature1', 'Feature2', 'MahalanobisDistance', 'IsOutlier']])